In [27]:
import numpy as np
import cv2 
from matplotlib import pyplot as plt

UBIT = 'mgosi'
np.random.seed(sum([ord(c) for c in UBIT]))

# Creating SIFT Images

In [28]:
img1 = cv2.imread('tsucuba_left.png')
img2 = cv2.imread('tsucuba_right.png')

gray_1= cv2.cvtColor(img1,cv2.COLOR_BGR2GRAY)
gray_2= cv2.cvtColor(img2,cv2.COLOR_BGR2GRAY)

sift = cv2.xfeatures2d.SIFT_create()

kp_1, dp_1 = sift.detectAndCompute(gray_1,None)
kp_2, dp_2 = sift.detectAndCompute(gray_2,None)

img_1=cv2.drawKeypoints(gray_1,kp_1,img1)
img_2=cv2.drawKeypoints(gray_2,kp_2,img2)

cv2.imwrite('task2_sift1.jpg',img_1)
cv2.imwrite('task2_sift2.jpg',img_2)

True

# Knn Matching

In [29]:
#Using Brute Force Matcher
bf = cv2.BFMatcher()
#tries to match the closed descriptor to the second image
matches = bf.knnMatch(dp_1, dp_2, k= 2)

# store all the good_matches matches as per Lowe's ratio test.
good_matches = []
pts1 = []
pts2 = []

for m,n in matches:
    if m.distance < 0.75*n.distance:
        good_matches.append(m)
        pts2.append(kp_2[m.trainIdx].pt)         #Gives us the index of the descriptor in the list of train descriptors 
        pts1.append(kp_1[m.queryIdx].pt)        #Gives us the index of the descriptor in the list of query descriptors 

img_3 = cv2.drawMatches(img_1,kp_1,img_2,kp_2,good_matches,None,flags=2)
cv2.imwrite("task2_matches_knn.jpg",img_3)

True

# Fundamental Matrix

In [30]:
pts1 = np.int32(pts1)
pts2 = np.int32(pts2)

#Selecting 10 random points
ran_pts1 = []
ran_pts2 = []
for i in [np.random.randint(0,len(pts1) -1 ) for x in range(10)] :
    ran_pts1.append(pts1[i])
    ran_pts2.append(pts2[i])

ran_pts1 = np.int32(ran_pts1)
ran_pts2 = np.int32(ran_pts2)

#print (ran_pts1, ran_pts2)
F, mask = cv2.findFundamentalMat(pts1,pts2,cv2.FM_RANSAC)

# We select only inlier points
pts1 = pts1[mask.ravel()==1]
pts2 = pts2[mask.ravel()==1]
print ("Fundamental Matrix :")
print (F)

Fundamental Matrix :
[[-2.12607354e-06 -8.10713687e-05  7.47530309e-02]
 [ 4.60726414e-05  3.79326900e-05  1.32728554e+00]
 [-7.52042326e-02 -1.32608913e+00  1.00000000e+00]]


In [31]:
def drawlines(img1,img2,lines,pts1,pts2):
    r,c = img1.shape[:2]
    img1 = cv2.cvtColor(img1,cv2.COLOR_GRAY2BGR)
    img2 = cv2.cvtColor(img2,cv2.COLOR_GRAY2BGR)

    for r,pt1,pt2, clr in zip(lines,pts1,pts2, color):
        x0,y0 = map(int, [0, -r[2]/r[1] ])
        x1,y1 = map(int, [c, -(r[2]+r[0]*c)/r[1] ])
        img1 = cv2.line(img1, (x0,y0), (x1,y1), clr,1)
        img1 = cv2.circle(img1,tuple(pt1),5,clr,-1)
        img2 = cv2.circle(img2,tuple(pt2),5,clr,-1)
    return img1,img2

# Epipolar Images

In [24]:
#Taking 10 random 
color = []
for i in range(10):
    color.append(tuple(np.random.randint(0,255,3).tolist()))
#Find epilines corresponding to points in right image (second image) and
# drawing its lines on left image
lines1 = cv2.computeCorrespondEpilines(ran_pts2.reshape(-1,1,2), 2,F)
lines1 = lines1.reshape(-1,3)
img5,img6 = drawlines(gray_1,gray_2,lines1,ran_pts1,ran_pts2)

#Find epilines corresponding to points in left image (first image) and
# drawing its lines on right image
lines2 = cv2.computeCorrespondEpilines(ran_pts1.reshape(-1,1,2), 1,F)
lines2 = lines2.reshape(-1,3)
img3,img4 = drawlines(gray_2,gray_1,lines2,ran_pts2,ran_pts1)

cv2.imwrite("task2_epi_right.jpg",img5)
cv2.imwrite("task2_epi_left.jpg",img3)

plt.show()

# Disparity

In [25]:
#Calculating Disparity

window_size = 3
min_disp = 16
num_disp = 144-min_disp
#setting the parameters to show disparity
stereo = cv2.StereoSGBM_create(minDisparity = min_disp,
        numDisparities = num_disp,
        uniquenessRatio = 10,
        speckleWindowSize = 100,
        speckleRange = 32,
        disp12MaxDiff = 1,
        P1 = 8*3*window_size**2,
        P2 = 32*3*window_size**2
    )
disparity = stereo.compute(gray_1,gray_2).astype(np.float32)/16.0
cv2.imwrite('task2_disparity.jpg',disparity)

True